In [ ]:
import sys
import itertools
from pathlib import Path
from os import getcwd
sys.path.append(str(Path(getcwd()).parent))
from os.path import join, basename

import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 1000)

from bulletin import default_input, default_output
from bulletin.systems import Notifica, CasosConfirmados
from bulletin.utils import utils, normalize
from bulletin.utils.static import Municipios
from bulletin.utils.xls_ import fit_cols

import random
import gc
from tqdm import tqdm

In [ ]:
from datetime import datetime, date, timedelta

today = pd.to_datetime(date.today())
ontem = today - timedelta(1)
anteontem = ontem - timedelta(1)

In [ ]:

notifica = Notifica('notifica')
notifica.load(compress=False)
# notifica.df = notifica.df.loc[notifica.casos_confirmados_mask()]
# notifica.df = notifica.df.sample(frac=0.01)

In [ ]:
# notifica.df.set_index('id', inplace=True)
notifica.df.reset_index('id', inplace=True)

In [ ]:
notifica.df.groupby('classificacao_final')[['sexo']].count()

In [ ]:
# 
# notifica.normalize()
# notifica.df

In [ ]:
# 
# aDtD = notifica.analise_data_diagnostico()
# aDtD.to_pickle('analise_data_diagnostico.pkl')
# aDtD.groupby('data_min_distance')[['idx_min_distance']].count().plot()

In [ ]:
# fichas_com_datas_distantes = aDtD.loc[aDtD['diff_min_max']>30]
# idx_fichas_com_datas_distantes = list(fichas_com_datas_distantes.index)
# fichas_com_datas_distantes

In [ ]:
for col in [ col for col in notifica.df.columns if 'duplicated_' in col ]:
    del notifica.df[col]

In [ ]:
notifica.df

In [ ]:
# 
duplicados = notifica.check_duplicates()
duplicados = duplicados.drop_duplicates('ids_str')
# duplicados.to_pickle('analise_duplicados.pkl')
duplicados

In [ ]:
duplicados['ids']

In [ ]:
aDtD = pd.read_pickle('analise_data_diagnostico.pkl')
aDtD

In [ ]:
duplicados = pd.read_pickle('analise_duplicados.pkl')
duplicados

In [ ]:
# [item for sublist in duplicados['ids'].values.tolist() for item in sublist]

In [ ]:

duplicados.groupby('duplicated_ncols')[['duplicated']].count()

In [ ]:
duplicados.groupby(['duplicated_ncols','duplicated_cols'])[['duplicated']].count()#.sort_values(by='duplicated')

In [ ]:
# raise
# duplicados = duplicados.loc[(duplicados['duplicated_ncols']>=2)|duplicados['duplicated_cols'].str.contains('hash_nasc')]

In [ ]:
# notifica.df.set_index('id', inplace=True)
notifica.df = notifica.df.loc[(
            (notifica.df['classificacao_final']==2) & 
            (notifica.df['status_notificacao'].isin([1,2])) & 
            (notifica.df['excluir_ficha']==2))
            ]
notifica.df

In [ ]:
# notifica.df.loc[notifica.df['id']==71379]
# duplicados
# criterio2 = (
#             (notifica.df['classificacao_final']==2) & 
#             (notifica.df['status_notificacao'].isin([1,2])) & 
#             (notifica.df['excluir_ficha']==2) & 
#             (notifica.df['hash_nasc']=='ELIANADASILVA08031977') & 
#             notifica.df[col].duplicated(keep=keep) &
#             notifica.df[col].notna()
#     )

# notifica.df.loc[criterio2]


In [ ]:
try:
    del (notifica.df['duplicated'])
except:
        pass
    
try:
    del notifica.df['duplicated_cols']
except:
    pass

try:
    del notifica.df['duplicated_ids']
except:
    pass

columns=['hash_nasc']
keep=False

notifica.df.insert(len(notifica.df.columns),'duplicated',False)
notifica.df.insert(len(notifica.df.columns),'duplicated_cols','')
notifica.df.insert(len(notifica.df.columns),'duplicated_ids','')



for col in columns:
    criterio = (
            notifica.df[col].notna() &
            notifica.df[col].duplicated(keep=False)
        )
    
    
    display(notifica.df.loc[criterio,col])

    notifica.df.loc[criterio,'duplicated'] = True
    notifica.df.loc[criterio,f"duplicated_cols"] =  notifica.df.loc[criterio,'duplicated_cols'].apply( lambda l: utils.strlist(l,col) )
    notifica.df.loc[criterio,f"{col}_duplicated_ids"] = notifica.df.loc[criterio,[col,'sexo']].groupby([col]).transform(lambda x: ',' if len(x) == 0 else ",".join(x.astype(str).values))

# duplicados = notifica.df.loc[notifica.df['duplicated'],['duplicated','duplicated_cols'] + sorted([ col for col in notifica.df.columns if 'cns' in col or 'cpf' in col or 'hash' in col ]) ].set_index('id')

# display(duplicados)

# group_duplicados = duplicados[ [ col for col in duplicados.columns if 'ids' in col ]].fillna('0')

# display(group_duplicados)

# group_duplicados.iloc[:,:-1] += ','

# display(group_duplicados)

# group_duplicados['ids'] = group_duplicados.sum(1).str.split(',').apply(lambda x: sorted(list(map(int,set(x) - {'0'}))))

# display(group_duplicados)

# duplicados = pd.merge(duplicados[['duplicated','duplicated_cols']],group_duplicados[['ids']],left_index=True,right_index=True,how='left')

# display(duplicados)

# duplicados['ids_len'] = duplicados['ids'].apply(len)

# display(duplicados)

# duplicados['ids_str'] = duplicados['ids'].apply(lambda x: ",".join(map(str,x)))

# display(duplicados)

# duplicados['duplicated_ncols'] = duplicados['duplicated_cols'].apply(lambda x: len(x.split(',')))

# display(duplicados)

# # for col in [ col for col in notifica.df.columns if 'duplicated_' in col ]:
# #     del notifica.df[col]


In [ ]:
aDtD.loc[duplicados.iloc[1]['ids']]

In [ ]:

# notifica.df.set_index('id',inplace=True)
duplicados = duplicados.sort_index()
#### HERE !!!!!!!!!!!!!!!!!!!
notifica.df['first_infect'] = False
notifica.df['socorro_deus'] = False
notifica.df['reinfect'] = False
notifica.df['days_between'] = -1
notifica.df['another_ids'] = None
notifica.df['duplicated_cols'] = None
notifica.df['duplicated_ncols'] = -1
notifica.df['block_cols_update'] = ''

for idx in tqdm(duplicados.index):
    row = duplicados.loc[idx]
    row_ids = [ id for id in row['ids'] if id not in idx_fichas_com_datas_distantes ]
    print(row_ids)
    notifica.df.loc[row_ids[0:],'first_infect'] = True
    if len(row_ids) > 1:
        notifica.df.loc[row_ids,['duplicated_cols','duplicated_ncols','another_ids']] = ( row['duplicated_cols'], row['duplicated_ncols'], row['ids_str'] )
        notifica.df.loc[row_ids[1:],'days_between'] = list(map(lambda x: x.astype('timedelta64[D]') / np.timedelta64(1, 'D'), notifica.df.loc[row_ids[1:],'data_diagnostico'].values - notifica.df.loc[row_ids[0:-1],'data_diagnostico'].values))
        notifica.df.loc[row_ids[1:],'days_between'] = notifica.df.loc[row_ids[1:],'days_between'].abs()
        
        index_mask_reifect = notifica.df.index.isin(row_ids[1:]) & (notifica.df['days_between'] >= 90)
        index_mask_not_reifect_neither_obito = notifica.df.index.isin(row_ids[1:]) & (notifica.df['days_between'] < 90) & (notifica.df['evolucao'] != 2)
        index_mask_not_reifect_but_obito = notifica.df.index.isin(row_ids[1:]) & (notifica.df['days_between'] < 90) & (notifica.df['evolucao'] == 2)

        notifica.df.loc[index_mask_reifect, 'reinfect'] = True
        
        notifica.df.loc[index_mask_not_reifect_but_obito, 'socorro_deus'] = True
        
        notifica.df.loc[index_mask_not_reifect_neither_obito, 'excluir_ficha'] = 1
        notifica.df.loc[index_mask_not_reifect_neither_obito, 'block_cols_update'] = notifica.df.loc[index_mask_not_reifect_neither_obito, 'block_cols_update'].apply(lambda l: utils.strlist(l,'excluir_ficha'))

In [ ]:
notifica.df['reinfect'].sum()

In [ ]:
notifica.df.loc[notifica.df['socorro_deus']].shape

In [ ]:
(notifica.df['excluir_ficha']==1).sum()

In [ ]:
notifica.df.loc[[item for sublist in duplicados['ids'] for item in sublist]]